# Auto-STED for imaging of FISH spot pairs

## How-To run:

1) Run the cell bellow ONCE if you (re)started the notebook

(to run a cell, click in it and press Ctrl-Enter)

In [1]:
from specpy import *
from Util.coordinates import *
from Util.datastructures import *
from Util.tile_util import *
from Util.imspector_util import *
from Util.dot_detection.Fiji_coordinate_detection import *
from Util.coordinate_util import *
from Util.dot_detection.Fiji_dummy import *
from Util.dot_detection.Fiji_coordinate_detection import read_coords
from Util.log_util import *
import pprint
from Util import datastructures, imspector_util
import time
from matplotlib import pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
from scipy import ndimage, spatial, stats
from spot_util import *
from display_util import *
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 10]

class FinishedException(Exception):
    def __init__(self, n_images, time_passed, *args):
        self.n_images = n_images
        self.time_passed = time_passed
        super(FinishedException, self).__init__(n_images, time_passed, *args)
        
def get_relative_coords(current_coords, coords):
    '''
    make all coordinates in coords relative to current_coords
    '''
    res = list()
    for c in coords:
        res.append(list(np.array(c, dtype=float) - np.array(current_coords,  dtype=float)))
    return res

def do_runs(outpaths, coords, current_coords, max_time, max_sted_images, logname='log.csv'):
    
    if len(outpaths) != len(coords):
        print('please specify an output path for every coordinate')
        return    
        
    for i in range(len(coords)):
        do_run(outpaths[i], coords[i], current_coords, max_time=max_time, max_sted_images=max_sted_images, first_z_size=2e-5, logname=logname)

def do_run(outpath, coords, current_coords, move_size = [6e-5, 6e-5], fov_size_ov = [5e-05, 5e-05, 0.7e-5], first_z_size= None,
           fov_size_sted = [3e-06, 3e-06, 1.4e-06], max_time = None, max_sted_images = None,
           ov_settings = 'C:/Users//RESOLFT/Desktop/ov.json', sted_settings = 'C:/Users//RESOLFT/Desktop/det.json',
           logname='log.csv'):

    
    
    # correct for global coordinates
    coords_corrected = get_relative_coords(current_coords, [coords])[0]
    #print(current_coords)
    #print(coords)
    #print(coords_corrected)
    #return
    
    # connect
    im = Imspector()
    
    # keep track of time and number of STED images taken
    start_time = time.time()
    n_sted_images = 0


    see = Settings()
    see.load_from_file(ov_settings)

    # coordinate generator (offset = 0)
    coords = df_circle_generator(move_size)

    #keep track of maximum z -> use that as next midplane
    zpos = 0.0;

    
    # setup output dir and name management
    d = os.path.dirname(outpath)
    if not os.path.exists(d):
        os.makedirs(d)
        
    # setup logging
    logfd = open(os.path.join(outpath, logname), 'w')
    logwriter = setup_csv_writer(logfd)
        
    nm = datastructures.NameManagement(d)
    nm.add_counter('field')
    nm.add_counter('sted')

    # measurement will be created on first run
    ms = None
    
    first_field = True

    
    for c in coords:

        try:
            
            # set the correct fov
            c.set_fov_len(fov_size_ov)
            
            # do a larger z stack at the first field
            if (first_z_size != None) and first_field:
                first_field = False
                fov_ = c.get_fov_len()
                fov_[2] = first_z_size
                c.set_fov_len(fov_)


            
            # set stage offset
            stage_old = c.get_bench_coords()
            stage_new = list(np.array(stage_old) + np.array(coords_corrected))
            c.set_bench_coords(stage_new)
            
            # z-correction
            offset_old = c.get_scan_offset()
            offset_old[2] = zpos
            c.set_offset_coords(offset_old)

            see.set_to_coordinates(c)

            # acquire image and save
            name = nm.get_next_image_name('field')
            acquire_measurement(im, see, ms)
            ms = im.active_measurement()
            ms.save_as(name)
            
            # log the acquisition
            
            stg_coords_for_log = list(np.array(c.get_bench_coords(), dtype=float) + np.array(current_coords, dtype=float))
            l = make_csv_line(name, stg_coords=stg_coords_for_log, scan_coords=c.get_scan_offset())
            logwriter.writerow(l)

            
            # find pairs
            sites = pair_finder(ms, thresholds = [0.9, 0.7], normalize=False)

            # get images in both channels
            # and plot the detections in max-projection
            stack1 = ms.stack(0).data()[0,:,:,:]
            stack2 = ms.stack(1).data()[0,:,:,:]
            draw_detections_2c(stack1, stack2, [s[-1::-1] for s in sites], [1, 10], 0, 3)
            plt.show()

            

            corner = middle2corner(c.get_scan_offset(), c.get_fov_len())
            pixelsd = get_pixel_size(ms)
            actual_coords = corner2spot(corner, sites, pixelsd)

              
            # check if we should stop
            time_passed = time.time() - start_time
                    
            ## check if time has run out
            if (max_time != None) and (time_passed > max_time):
                raise FinishedException(n_sted_images, time_passed)           


            # update focus
            zpos += focus_in_stack(ms.stack(0).data(), pixelsd[2])
            
            # inform about new focus and detected spots
            print('new z:' + str(zpos))
            print(actual_coords)

            co = c.copy()
            see2 = Settings()
            see2.load_from_file(sted_settings)

            # restart counting the sted images ( -> field1sted0 - filed1stedn, field2sted0 - field2stedn, ...)
            nm.reset_counter('sted')

            for i in actual_coords:
                
                # set the fov and spot coordinates
                co.set_fov_len(fov_size_sted)
                co.set_offset_coords(i)
                see2.set_to_coordinates(co)

                # do measurement and save
                name = nm.get_next_image_name('sted')
                acquire_measurement(im, see2, ms)
                ms = im.active_measurement()
                ms.save_as(name)  
                
                # log the acquisition
                stg_coords_for_log = list(np.array(c.get_bench_coords(), dtype=float) + np.array(current_coords, dtype=float))
                l = make_csv_line(name, stg_coords=stg_coords_for_log, type='sted', scan_coords=list(co.get_scan_offset()))
                logwriter.writerow(l)
                
                # check if we should stop
                time_passed = time.time() - start_time
                n_sted_images += 1
                
                ## check if we have enough images
                if (max_sted_images != None) and (n_sted_images >= max_sted_images):
                    raise FinishedException(n_sted_images, time_passed)
                    
                ## check if time has run out
                if (max_time != None) and (time_passed > max_time):
                    raise FinishedException(n_sted_images, time_passed)

                    
        except FinishedException as e:
            print('RUN FINISHED. Acquired ' + str(e.n_images) + ' STED images in ' + str(e.time_passed) + 's.')
            logfd.close()
            break
            
        except KeyboardInterrupt:
            print('RUN STOPPED BY USER')
            logfd.close()
            break



# OPTION 1: Measurement at current position 
2) output path in the cell below
3) run the cell (Imspector has to be running and Tools->Run server has to be ckecked in Imspector)

# Stopping the run
4) click the "STOP"-button at the top of this window
5) the current measurement in Imspector will finish, but won't be saved

In [ ]:
# where to save results
outpath = 'C:/Users//RESOLFT/Desktop/AUTOMATION/5th_shipment_20161028/mixed_150s_A/GM_150sec_A/'

# the maximum number of sted images to acquire
max_sted_images = 500

# the maximum time to spend in one area (in SECONDS)
max_time = 12 * 60 * 60 # 12hrs

do_run(outpath, [0, 0, 0], max_time=max_time, max_sted_images=max_sted_images)


# OPTION 2: Measurement at multiple coordinates
2) insert coordinates and result paths in the cell below
3) run the cell (Imspector has to be running and Tools->Run server has to be ckecked in Imspector)

# Stopping the run
4) click the "STOP"-button at the top of this window
5) the current measurement in Imspector will finish, but won't be saved

In [ ]:
# GLOBAL stage coordinates of where to do scans (in METERS!)
coords = [[11077.9e-6, -2195.5e-6, 1903.2e-6],
          [2742.5e-6, -1927.1e-6, 1904.6e-6]]

# the current global stage coordinates (in METERS!)
current_coords = [2742.5e-6, -1927.1e-6, 1904.6e-6]

# for each coordinate: where to save results
outpaths = ['C:/Users//RESOLFT/Desktop/AUTOMATION/6th_shipment_20161219/mixed_HBG2_HS2_B/GM_180sec_B/',
            'C:/Users//RESOLFT/Desktop/AUTOMATION/6th_shipment_20161219/mixed_HBG2_HS2_B/K562_180sec_B/']

# the maximum number of sted images to acquire
max_sted_images = 500

# the maximum time to spend in one area (in SECONDS)
max_time = 12 * 60 * 60 # 12hrs

## do the runs
do_runs(outpaths, coords, current_coords, max_time, max_sted_images)

# code below here is only for testing purposes

In [ ]:
im = Imspector()
ms = im.active_measurement()
stack1 = ms.stack(0).data()[0,:,:,:]
stack2 = ms.stack(1).data()[0,:,:,:]
stack1 = np.array(stack1, np.float)
stack2 = np.array(stack2, np.float)

sites = pair_finder_inner(stack1, stack2, 3, threshold = 0.8, invertAxes=True, normalize=False)
draw_detections_2c(stack1, stack2, [s[-1::-1] for s in sites], [1, 10], 0)
print(sites)


In [ ]:
if 0:
    print(1)

In [ ]:
stack1.shape

In [ ]:
-0.0046678 - 0.0106896

In [ ]:
pair_finder(Imspector().active_measurement())